In [1]:
import pandas as pd
import numpy as np

z = pd.read_csv(r"C:\Users\gromo\OneDrive\Documents\transactions_diy.csv")

# загружаем датасет

In [2]:
# смотрим датасет
z['Sale'] = z['Quantity'] * z['Amount']
z

,TrDte,BCode,ClientID,Item,ItemGroup,Quantity,Amount,Sale
0,01.09.2017,code000000001,client13166,sku8444,Скобяные изделия,1,29,29
1,01.09.2017,code000000001,client13166,sku12545,Оборудование для сада и дачи,1,329,329
2,01.09.2017,code000000001,client13166,sku3391,Инструменты,1,169,169
3,01.09.2017,code000000001,client13166,sku20444,Инструменты,2,578,1156
4,01.09.2017,code000000002,client1239,sku29959,Скобяные изделия,1,329,329
...,...,...,...,...,...,...,...,...
1008683,31.10.2019,code000290228,client25258,sku5837,Скобяные изделия,1,89,89
1008684,31.10.2019,code000290228,client25258,sku26161,Стойматериалы,2,598,1196
1008685,31.10.2019,code000290228,client25258,sku20658,Сантехника,1,159,159
1008686,31.10.2019,code000290229,client22472,sku16069,Столярные изделия,1,85,85


In [3]:
# смотрим какие типы данных

z.dtypes

TrDte        object
BCode        object
ClientID     object
Item         object
ItemGroup    object
Quantity      int64
Amount        int64
Sale          int64
dtype: object

In [4]:
# замечаем, что TrDte - object


In [5]:
# Для первого задания мы создадим отдельный датафрейм с сортировкой ItemGroup по количеству строк
# Полученный результат отсортируем по столбцу Amount по убыванию
# Выведем строку с индексом 0 и поймем самую популярную группу (начало нумирации - 0)

z1 = z.groupby(['ItemGroup']).count()
z1 = z1.sort_values(by=['BCode'], ascending=[0])
z1 = z1['Quantity']
z1[0]




C:\Users\gromo\AppData\Local\Temp\ipykernel_15312\3628628872.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  z1[0]


161877

In [6]:
# Во втором задании группируем исходный датафрейм по столбцу ClientID и считаем сумму Amount
# Ставим фильтр на сводную таблицу < 100
# смотрим количество строк у отфильтрованной таблицы
# выводим длину

z2 = z.groupby(['ClientID']).sum(['Amount'])
z2 = z2.loc[z2['Amount'] <= 100]
z2.shape[0]

401

In [7]:
# Для третьего задания группируем исходный датафрейм по ClientID и считаем количество ClientId, а также сумму по Amount
# Фильтруем по кол-ву ClientID = 1
# По полученным данным считаем среднее по сумме Amount
# Округляем до ближайшего целого

z3 = z
z3['count_dates'] = z3.groupby(['ClientID'])['TrDte'].transform('nunique')
z3['sum_amount'] = z3.groupby(['ClientID'])['Amount'].transform('sum')
z3 = z3[z3['count_dates'] == 1]
z3 = z3.drop_duplicates(subset='ClientID')
z3['sum_amount'].mean()

3535.44785753169

In [8]:
# Добавляем столбцы через оконные функции в новый датафрейм
# считаем корреляцию методов corr() и пишем метод 'pearson


z4 = z
z4['sum_amount'] = z4.groupby(['ClientID'])['Amount'].transform('sum')
z4['sum_quantity'] = z4.groupby(['ClientID'])['Quantity'].transform('sum')
z4['count_dates'] = z4.groupby(['ClientID'])['TrDte'].transform('nunique')
z4['count_items'] = z4.groupby(['ClientID'])['Item'].transform('nunique')

z4 = z4[['sum_amount', 'count_items']]
z4[['sum_amount', 'count_items']].corr(method='pearson')


,sum_amount,count_items
sum_amount,1.000000,0.615227
count_items,0.615227,1.000000


In [9]:
# Делаем датасет из двух колонок
# Делаем новый столбец по кол-ву уникальных дат у клиентов
# Ставим условие > 50 уникальных дат


z5 = z[['ClientID', 'TrDte']]
z5['trdte_count'] = z5.groupby(['ClientID'])['TrDte'].transform('nunique')
z5 = z5[z5['trdte_count'] > 50]
z5 = z5.drop_duplicates(subset='ClientID')
z5

C:\Users\gromo\AppData\Local\Temp\ipykernel_15312\485876498.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z5['trdte_count'] = z5.groupby(['ClientID'])['TrDte'].transform('nunique')


,ClientID,TrDte,trdte_count
910,client20512,04.09.2017,70
1498,client22177,06.09.2017,65
1618,client34917,06.09.2017,65
1916,client9108,08.09.2017,64
2056,client3082,08.09.2017,100
...,...,...,...
554946,client11553,09.03.2019,63
570111,client30155,18.03.2019,69
608991,client17811,09.04.2019,54
663352,client5900,01.05.2019,60


In [10]:
# Переносим исходный датасет в отдельный
# Добавляем столбец суммы кол-ва продуктов по всем транзацкциям
# проверяем на != 0
# считаем среднее по транзации
# считаем среднее по средним транзакций
# Проверяем на <= 10

z6 = z
z6['sum_quantity'] = z6.groupby(['ClientID'])['Quantity'].transform('sum')
z6 = z6[z6['sum_quantity'] != 0]
z6['real_mean'] = z6['Amount'] / z6['Quantity']
z6['mean_amount'] = z6.groupby(['Item'])['real_mean'].transform('mean')
z6 = z6[z6['mean_amount'] <= 10]
z6 = z6.drop_duplicates(subset='Item')
z6



,TrDte,BCode,ClientID,Item,ItemGroup,Quantity,Amount,Sale,count_dates,sum_amount,sum_quantity,count_items,real_mean,mean_amount
82,01.09.2017,code000000031,client28103,sku27292,Стойматериалы,20,160,3200,1,1098,22,3,8.000000,8.268076
247,02.09.2017,code000000062,client684,sku1643,Электротовары,3,28,84,25,41396,248,63,9.333333,9.088940
324,02.09.2017,code000000083,client41232,sku22202,Комнатные растения и цветы,8,39,312,5,4935,112,62,4.875000,4.986111
640,03.09.2017,code000000162,client28304,sku19020,Сантехника,1,8,8,4,5782,44,20,8.000000,9.743871
641,03.09.2017,code000000162,client28304,sku5084,Сантехника,5,35,175,4,5782,44,20,7.000000,8.484430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828711,25.07.2019,code000231918,client32351,sku894,Товары для дачи и отдыха,1,9,9,5,13499,47,19,9.000000,9.000000
836680,29.07.2019,code000234469,client41916,sku15117,Товары для дачи и отдыха,5,43,215,10,12074,149,122,8.600000,8.650000
877030,20.08.2019,code000247598,client25251,sku16306,Комнатные растения и цветы,3,9,27,9,7475,68,42,3.000000,3.000000
879578,21.08.2019,code000248382,client11968,sku24062,Оборудование для сада и дачи,1,10,10,23,40858,229,121,10.000000,9.750000
